# 📋 9주차: AI 애플리케이션 배포와 모니터링  
## 🎯 과제 핵심 목표 - 8주차 파이프라인을 실제 서비스로 배포하기  
  
### FastAPI로 API 만들기  
### 클라우드에 배포하기  
- 배포된 URL 2개 확인:  
  - API 엔드포인트: https://yourapp.railway.app/api/v1/...  
  - Swagger UI: https://yourapp.railway.app/docs  
- LangSmith로 모니터링하기  

## Milestone 1: FastAPI 앱 만들기 (40%)
### Section 1: 기본 구조
  - main.py (FastAPI 앱)
  - models.py (Pydantic)
  - 8주차 파이프라인 재사용

### Section 2: 3개 엔드포인트
  - POST /api/v1/run
  - GET /api/v1/status/{run_id}
  - GET /api/v1/result/{run_id}

### Section 3: 로컬 테스트
  - uvicorn으로 실행
  - http://localhost:8000/docs 확인
## Milestone 2: 클라우드 배포 (40%)
### Section 4: Railway 배포
  - requirements.txt
  - 환경변수 설정
  - GitHub 연동

### Section 5: 배포 확인
  - https://yourapp.railway.app/docs
  - API 호출 테스트
## Milestone 3: LangSmith 모니터링 (20%)
### Section 6: LangSmith 연동
  - 환경변수 설정
  - 트레이스 확인
  - 스크린샷 3장
  
## 📁 프로젝트 구조 (최소)  
week9-api-deployment/  
├── app/  
│   ├── main.py              # FastAPI 앱  
│   ├── models.py            # Pydantic 모델  
│   └── pipeline.py          # 8주차 파이프라인 (간소화)  
├── .env  
├── requirements.txt  
└── README.md  

In [6]:
import os
from pathlib import Path

PROJECT_ROOT = Path("week9-api-deployment")
PROJECT_ROOT.mkdir(exist_ok=True)

(PROJECT_ROOT / "app").mkdir(exist_ok=True)

print("✅ 프로젝트 폴더 생성 완료")
print(f"📁 {PROJECT_ROOT}/")
print(f"📁 {PROJECT_ROOT}/app/")

✅ 프로젝트 폴더 생성 완료
📁 week9-api-deployment/
📁 week9-api-deployment/app/


In [2]:
poetry_lock_path = Path("poetry.lock")
requirements_txt_path = PROJECT_ROOT / "requirements.txt"

if poetry_lock_path.exists():
    requirements = []
    with poetry_lock_path.open(encoding="utf-8") as f:
        in_package = False
        for line in f:
            if line.strip() == "[[package]]":
                in_package = True
                name, version = None, None
            elif in_package:
                if line.startswith("name ="):
                    name = line.split("=", 1)[1].strip().strip('"')
                elif line.startswith("version ="):
                    version = line.split("=", 1)[1].strip().strip('"')
                elif line.strip() == "":
                    if name and version:
                        requirements.append(f"{name}=={version}")
                    in_package = False
        # 마지막 패키지 처리
        if in_package and name and version:
            requirements.append(f"{name}=={version}")

    with requirements_txt_path.open("w", encoding="utf-8") as f:
        f.write("\n".join(requirements))
    print(f"✅ requirements.txt 생성 완료: {requirements_txt_path}")
else:
    print("❌ poetry.lock 파일이 존재하지 않습니다.")

✅ requirements.txt 생성 완료: week9-api-deployment/requirements.txt


In [3]:
models_code = '''from pydantic import BaseModel, Field
from typing import Optional, List, Dict, Any
from datetime import datetime

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 요청 모델
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

class CreateJobRequest(BaseModel):
    """파이프라인 실행 요청"""
    user_goal: str = Field(..., description="사용자 요청", example="AI 윤리 에세이")
    
    class Config:
        json_schema_extra = {
            "example": {
                "user_goal": "AI 윤리에 대한 300자 에세이"
            }
        }

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 응답 모델
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

class JobStatusResponse(BaseModel):
    """작업 상태 응답"""
    run_id: str
    status: str  # pending, running, completed, failed
    current_step: int
    completed_steps: List[int]
    created_at: str
    
    class Config:
        json_schema_extra = {
            "example": {
                "run_id": "run_abc123",
                "status": "running",
                "current_step": 2,
                "completed_steps": [1, 2],
                "created_at": "2026-01-11T12:00:00"
            }
        }

class JobResultResponse(BaseModel):
    """작업 결과 응답"""
    run_id: str
    status: str
    user_goal: str
    final_output: Optional[str] = None
    step_outputs: Optional[Dict[str, Any]] = None
    
    class Config:
        json_schema_extra = {
            "example": {
                "run_id": "run_abc123",
                "status": "completed",
                "user_goal": "AI 윤리 에세이",
                "final_output": "AI 윤리는..."
            }
        }

class ErrorResponse(BaseModel):
    """에러 응답"""
    error: str
    detail: Optional[str] = None
'''

(PROJECT_ROOT / "app" / "models.py").write_text(models_code)

print("✅ app/models.py 생성")

✅ app/models.py 생성


In [4]:
pipeline_code = '''import os
from typing import TypedDict, List, Dict, Any
from datetime import datetime
import uuid
from pathlib import Path

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, START, END

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# State 정의 (8주차 재사용)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

class PipelineState(TypedDict):
    run_id: str
    user_goal: str
    current_step: int
    completed_steps: List[int]
    step_outputs: Dict[str, Any]
    status: str
    timestamp: str

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# LLM 설정
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 간소화된 노드 (실제로는 8주차 4개 Step 사용)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def step_1_node(state: PipelineState) -> PipelineState:
    """Step 1: 요구사항 분석"""
    print(f"[Step 1] 분석 중: {state['user_goal']}")
    
    prompt = f"다음 요청을 분석하세요: {state['user_goal']}"
    result = llm.invoke(prompt).content
    
    step_outputs = state.get("step_outputs", {})
    step_outputs["step_1"] = {"analysis": result}
    
    return {
        "current_step": 1,
        "completed_steps": [1],
        "step_outputs": step_outputs,
        "status": "running",
        "timestamp": datetime.now().isoformat()
    }

def step_2_node(state: PipelineState) -> PipelineState:
    """Step 2: 콘텐츠 생성"""
    print(f"[Step 2] 생성 중")
    
    analysis = state["step_outputs"]["step_1"]["analysis"]
    prompt = f"다음 분석을 바탕으로 콘텐츠를 작성하세요:\\n{analysis}"
    result = llm.invoke(prompt).content
    
    step_outputs = state["step_outputs"]
    step_outputs["step_2"] = {"content": result}
    
    return {
        "current_step": 2,
        "completed_steps": [1, 2],
        "step_outputs": step_outputs,
        "status": "completed",
        "timestamp": datetime.now().isoformat()
    }

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 그래프 구성
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

graph_builder = StateGraph(PipelineState)
graph_builder.add_node("step_1", step_1_node)
graph_builder.add_node("step_2", step_2_node)
graph_builder.add_edge(START, "step_1")
graph_builder.add_edge("step_1", "step_2")
graph_builder.add_edge("step_2", END)

pipeline_graph = graph_builder.compile()

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 헬퍼 함수
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def create_initial_state(user_goal: str) -> PipelineState:
    """초기 State 생성"""
    return {
        "run_id": "run_" + uuid.uuid4().hex[:8],
        "user_goal": user_goal,
        "current_step": 0,
        "completed_steps": [],
        "step_outputs": {},
        "status": "pending",
        "timestamp": datetime.now().isoformat()
    }

def run_pipeline(user_goal: str) -> PipelineState:
    """파이프라인 실행"""
    state = create_initial_state(user_goal)
    result = pipeline_graph.invoke(state)
    return {**state, **result}
'''

(PROJECT_ROOT / "app" / "pipeline.py").write_text(pipeline_code)

print("✅ app/pipeline.py 생성 (8주차 간소화)")

✅ app/pipeline.py 생성 (8주차 간소화)


In [5]:
main_code = '''from fastapi import FastAPI, HTTPException, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from dotenv import load_dotenv
import os
from typing import Dict
from datetime import datetime

from .models import (
    CreateJobRequest,
    JobStatusResponse,
    JobResultResponse,
    ErrorResponse
)
from .pipeline import run_pipeline, PipelineState

# 환경변수 로드
load_dotenv()

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# FastAPI 앱 생성
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

app = FastAPI(
    title="Week 9 Pipeline API",
    version="1.0.0",
    description="8주차 파이프라인을 RESTful API로 배포",
    docs_url="/docs",
    redoc_url="/redoc"
)

# CORS 설정
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 배포 시 특정 도메인으로 제한
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 인메모리 저장소 (간단한 구현)
# 실제 프로덕션에서는 Redis나 DB 사용
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

jobs: Dict[str, PipelineState] = {}

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 백그라운드 작업 함수
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def execute_pipeline(run_id: str, user_goal: str):
    """백그라운드에서 파이프라인 실행"""
    try:
        jobs[run_id]["status"] = "running"
        result = run_pipeline(user_goal)
        jobs[run_id] = {**jobs[run_id], **result}
    except Exception as e:
        jobs[run_id]["status"] = "failed"
        jobs[run_id]["error"] = str(e)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# API 엔드포인트
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

@app.get("/")
async def root():
    """루트 엔드포인트"""
    return {
        "message": "Week 9 Pipeline API",
        "docs": "/docs",
        "endpoints": {
            "create_job": "POST /api/v1/run",
            "get_status": "GET /api/v1/status/{run_id}",
            "get_result": "GET /api/v1/result/{run_id}"
        }
    }

@app.get("/health")
async def health():
    """헬스 체크"""
    return {"status": "healthy", "timestamp": datetime.now().isoformat()}

@app.post(
    "/api/v1/run",
    response_model=JobStatusResponse,
    summary="파이프라인 실행",
    description="새로운 파이프라인 작업을 시작합니다",
    responses={
        200: {"description": "작업 생성 성공"},
        500: {"model": ErrorResponse}
    }
)
async def create_job(
    request: CreateJobRequest,
    background_tasks: BackgroundTasks
):
    """파이프라인 실행 시작"""
    from .pipeline import create_initial_state
    
    # 초기 State 생성
    state = create_initial_state(request.user_goal)
    run_id = state["run_id"]
    
    # 저장
    jobs[run_id] = state
    
    # 백그라운드 실행
    background_tasks.add_task(execute_pipeline, run_id, request.user_goal)
    
    return JobStatusResponse(
        run_id=run_id,
        status="pending",
        current_step=0,
        completed_steps=[],
        created_at=state["timestamp"]
    )

@app.get(
    "/api/v1/status/{run_id}",
    response_model=JobStatusResponse,
    summary="작업 상태 조회",
    description="실행 중인 작업의 상태를 확인합니다",
    responses={
        200: {"description": "상태 조회 성공"},
        404: {"model": ErrorResponse}
    }
)
async def get_status(run_id: str):
    """작업 상태 조회"""
    if run_id not in jobs:
        raise HTTPException(status_code=404, detail="Job not found")
    
    job = jobs[run_id]
    
    return JobStatusResponse(
        run_id=run_id,
        status=job["status"],
        current_step=job["current_step"],
        completed_steps=job["completed_steps"],
        created_at=job["timestamp"]
    )

@app.get(
    "/api/v1/result/{run_id}",
    response_model=JobResultResponse,
    summary="작업 결과 조회",
    description="완료된 작업의 결과를 조회합니다",
    responses={
        200: {"description": "결과 조회 성공"},
        404: {"model": ErrorResponse},
        425: {"description": "작업이 아직 완료되지 않음"}
    }
)
async def get_result(run_id: str):
    """작업 결과 조회"""
    if run_id not in jobs:
        raise HTTPException(status_code=404, detail="Job not found")
    
    job = jobs[run_id]
    
    if job["status"] != "completed":
        raise HTTPException(
            status_code=425,
            detail=f"Job is still {job['status']}"
        )
    
    # 최종 출력 생성
    final_output = job["step_outputs"].get("step_2", {}).get("content", "")
    
    return JobResultResponse(
        run_id=run_id,
        status=job["status"],
        user_goal=job["user_goal"],
        final_output=final_output,
        step_outputs=job["step_outputs"]
    )
'''

(PROJECT_ROOT / "app" / "main.py").write_text(main_code)

print("✅ app/main.py 생성")
print("\n[구현된 엔드포인트]")
print("  POST /api/v1/run         - 파이프라인 실행")
print("  GET  /api/v1/status/{id} - 상태 조회")
print("  GET  /api/v1/result/{id} - 결과 조회")

✅ app/main.py 생성

[구현된 엔드포인트]
  POST /api/v1/run         - 파이프라인 실행
  GET  /api/v1/status/{id} - 상태 조회
  GET  /api/v1/result/{id} - 결과 조회


In [1]:
print("\n" + "="*80)
print("Cell 14: Health Check 테스트")
print("="*80)

import requests
import json

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 기본 설정
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

BASE_URL = "http://localhost:8000"

print(f"[테스트 대상] {BASE_URL}")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 1. 루트 엔드포인트 테스트
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n[1/2] 루트 엔드포인트 테스트: GET /")

try:
    response = requests.get(f"{BASE_URL}/")
    
    if response.status_code == 200:
        print(f"  ✅ 상태 코드: {response.status_code}")
        print(f"  응답:")
        print(json.dumps(response.json(), indent=2, ensure_ascii=False))
    else:
        print(f"  ❌ 상태 코드: {response.status_code}")
        
except requests.exceptions.ConnectionError:
    print(f"  ❌ 연결 실패!")
    print(f"  서버가 실행 중인지 확인하세요: {BASE_URL}")
    print(f"  터미널에서: poetry run uvicorn app.main:app --reload")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 2. Health Check 테스트
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n[2/2] Health Check: GET /health")

try:
    response = requests.get(f"{BASE_URL}/health")
    
    if response.status_code == 200:
        print(f"  ✅ 상태 코드: {response.status_code}")
        print(f"  응답:")
        print(json.dumps(response.json(), indent=2, ensure_ascii=False))
    else:
        print(f"  ❌ 상태 코드: {response.status_code}")
        
except requests.exceptions.ConnectionError:
    print(f"  ❌ 연결 실패!")

print("\n" + "="*80)
print("✅ 기본 엔드포인트 테스트 완료")
print("="*80)


Cell 14: Health Check 테스트
[테스트 대상] http://localhost:8000

[1/2] 루트 엔드포인트 테스트: GET /
  ✅ 상태 코드: 200
  응답:
{
  "message": "Week 9 Pipeline API",
  "docs": "/docs",
  "endpoints": {
    "create_job": "POST /api/v1/run",
    "get_status": "GET /api/v1/status/{run_id}",
    "get_result": "GET /api/v1/result/{run_id}"
  }
}

[2/2] Health Check: GET /health
  ✅ 상태 코드: 200
  응답:
{
  "status": "healthy",
  "timestamp": "2026-01-12T00:07:48.090731"
}

✅ 기본 엔드포인트 테스트 완료


In [2]:
print("\n" + "="*80)
print("Cell 15: POST /api/v1/run 테스트")
print("="*80)

import time

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 파이프라인 실행
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n[테스트] POST /api/v1/run")

request_data = {
    "user_goal": "AI 윤리에 대한 간단한 에세이"
}

print(f"\n요청 데이터:")
print(json.dumps(request_data, indent=2, ensure_ascii=False))

try:
    response = requests.post(
        f"{BASE_URL}/api/v1/run",
        json=request_data
    )
    
    if response.status_code == 200:
        print(f"\n✅ 상태 코드: {response.status_code}")
        
        result = response.json()
        run_id = result.get("run_id")
        
        print(f"\n응답:")
        print(json.dumps(result, indent=2, ensure_ascii=False))
        
        print(f"\n📝 run_id 저장: {run_id}")
        
        # 다음 테스트를 위해 저장
        globals()['test_run_id'] = run_id
        
    else:
        print(f"\n❌ 상태 코드: {response.status_code}")
        print(f"응답: {response.text}")
        
except Exception as e:
    print(f"\n❌ 에러 발생: {e}")

print("\n" + "="*80)
print("✅ POST /api/v1/run 테스트 완료")
print("="*80)


Cell 15: POST /api/v1/run 테스트

[테스트] POST /api/v1/run

요청 데이터:
{
  "user_goal": "AI 윤리에 대한 간단한 에세이"
}

✅ 상태 코드: 200

응답:
{
  "run_id": "run_8a866008",
  "status": "pending",
  "current_step": 0,
  "completed_steps": [],
  "created_at": "2026-01-12T00:45:41.642447"
}

📝 run_id 저장: run_8a866008

✅ POST /api/v1/run 테스트 완료


In [3]:
print("\n" + "="*80)
print("Cell 16: GET /api/v1/status/{run_id} 테스트")
print("="*80)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 상태 조회
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

if 'test_run_id' not in globals():
    print("⚠️ test_run_id가 없습니다. 먼저 Cell 15를 실행하세요.")
else:
    run_id = test_run_id
    
    print(f"\n[테스트] GET /api/v1/status/{run_id}")
    
    # 여러 번 조회 (파이프라인 진행 확인)
    for i in range(3):
        print(f"\n[조회 {i+1}/3]")
        
        try:
            response = requests.get(f"{BASE_URL}/api/v1/status/{run_id}")
            
            if response.status_code == 200:
                result = response.json()
                
                status = result.get("status")
                current_step = result.get("current_step")
                completed_steps = result.get("completed_steps")
                
                print(f"  상태: {status}")
                print(f"  현재 단계: Step {current_step}")
                print(f"  완료 단계: {completed_steps}")
                
                if status == "completed":
                    print(f"\n  ✅ 파이프라인 완료!")
                    break
                    
            else:
                print(f"  ❌ 상태 코드: {response.status_code}")
                
        except Exception as e:
            print(f"  ❌ 에러: {e}")
        
        # 잠시 대기
        if i < 2:
            print(f"  ⏳ 2초 대기 중...")
            time.sleep(2)

print("\n" + "="*80)
print("✅ GET /api/v1/status 테스트 완료")
print("="*80)


Cell 16: GET /api/v1/status/{run_id} 테스트

[테스트] GET /api/v1/status/run_8a866008

[조회 1/3]
  상태: completed
  현재 단계: Step 2
  완료 단계: [1, 2]

  ✅ 파이프라인 완료!

✅ GET /api/v1/status 테스트 완료


In [4]:
print("\n" + "="*80)
print("Cell 17: GET /api/v1/result/{run_id} 테스트")
print("="*80)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 결과 조회
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

if 'test_run_id' not in globals():
    print("⚠️ test_run_id가 없습니다. 먼저 Cell 15를 실행하세요.")
else:
    run_id = test_run_id
    
    print(f"\n[테스트] GET /api/v1/result/{run_id}")
    
    # 완료될 때까지 대기 (최대 30초)
    max_attempts = 15
    
    for attempt in range(max_attempts):
        try:
            # 먼저 상태 확인
            status_response = requests.get(f"{BASE_URL}/api/v1/status/{run_id}")
            
            if status_response.status_code == 200:
                status_data = status_response.json()
                status = status_data.get("status")
                
                if status == "completed":
                    print(f"\n✅ 파이프라인 완료! 결과 조회 중...")
                    
                    # 결과 조회
                    result_response = requests.get(f"{BASE_URL}/api/v1/result/{run_id}")
                    
                    if result_response.status_code == 200:
                        result = result_response.json()
                        
                        print(f"\n" + "="*80)
                        print("최종 결과")
                        print("="*80)
                        
                        print(f"\nrun_id: {result.get('run_id')}")
                        print(f"status: {result.get('status')}")
                        print(f"user_goal: {result.get('user_goal')}")
                        
                        final_output = result.get('final_output', '')
                        
                        print(f"\n[최종 출력]")
                        print("-" * 80)
                        print(final_output)
                        print("-" * 80)
                        
                        # step_outputs도 출력
                        step_outputs = result.get('step_outputs', {})
                        if step_outputs:
                            print(f"\n[단계별 출력]")
                            for step_name, step_data in step_outputs.items():
                                print(f"\n{step_name}:")
                                print(json.dumps(step_data, indent=2, ensure_ascii=False)[:200] + "...")
                        
                        break
                    else:
                        print(f"❌ 결과 조회 실패: {result_response.status_code}")
                        break
                        
                elif status == "failed":
                    print(f"\n❌ 파이프라인 실패!")
                    break
                    
                else:
                    print(f"  ⏳ 대기 중... (상태: {status}, 시도: {attempt+1}/{max_attempts})")
                    time.sleep(2)
                    
        except Exception as e:
            print(f"❌ 에러: {e}")
            break
    
    else:
        print(f"\n⚠️ 시간 초과 (30초)")

print("\n" + "="*80)
print("✅ GET /api/v1/result 테스트 완료")
print("="*80)


Cell 17: GET /api/v1/result/{run_id} 테스트

[테스트] GET /api/v1/result/run_8a866008

✅ 파이프라인 완료! 결과 조회 중...

최종 결과

run_id: run_8a866008
status: completed
user_goal: AI 윤리에 대한 간단한 에세이

[최종 출력]
--------------------------------------------------------------------------------
# AI 윤리에 대한 간단한 에세이

## 서론
인공지능(AI)의 발전은 지난 수십 년 동안 급속도로 이루어졌으며, 이는 우리의 생활 방식을 변화시키고 있습니다. AI는 의료, 교육, 금융 등 다양한 분야에서 혁신을 가져오고 있으며, 그 사회적 영향력은 날로 증가하고 있습니다. 그러나 이러한 발전과 함께 AI의 윤리적 문제도 부각되고 있습니다. AI 윤리는 기술이 인간 사회에 미치는 영향을 고려하여, 공정하고 안전하며 책임감 있는 방식으로 AI를 개발하고 사용하는 것을 목표로 합니다. 따라서 AI 윤리는 현대 사회에서 그 중요성이 점점 더 커지고 있습니다.

## AI 윤리의 정의
AI 윤리는 인공지능 시스템의 개발과 사용에서 발생할 수 있는 윤리적 고려 사항을 다루는 분야입니다. 이는 AI가 사람의 삶에 미치는 영향, 즉 공정성, 프라이버시, 책임, 투명성 등의 문제를 포함합니다. AI 윤리가 중요한 이유는, AI 시스템이 인간의 결정을 대체하거나 보조하는 데 사용되기 때문에, 이러한 시스템이 윤리적으로 설계되지 않으면 사회적 불평등이나 개인의 권리가 침해될 수 있기 때문입니다.

## 주요 윤리적 문제
AI 윤리에서 가장 중요한 문제 중 하나는 **편향과 공정성**입니다. AI 시스템은 주어진 데이터를 기반으로 학습하므로, 데이터에 포함된 편향이 결과에 직접적인 영향을 미칠 수 있습니다. 예를 들어, 특정 인종이나 성별에 대한 편향이 있는 데이터로 학습한 AI는 불공정한 결정을 내

  
## Section 3 목표

---

로컬에서 작동하는 API를 클라우드에 배포하여:

- ✅ 전 세계 어디서나 접근 가능하게 만들기  
- ✅ 배포된 URL 2개 확보:  
  1. **API 엔드포인트:** `https://yourapp.railway.app/api/v1/...`  
  2. **Swagger UI:** `https://yourapp.railway.app/docs`  

---

### 배포 플랫폼: Railway

---

**Railway를 선택하는 이유:**

- ✅ GitHub 연동 자동 배포  
- ✅ 무료 크레딧 제공 ($5)  
- ✅ 간단한 환경변수 설정  
- ✅ 자동 HTTPS 제공  
- ✅ 빠른 배포 속도  

**대안:**  
- Render (무료 티어, 느린 콜드 스타트)  
- Fly.io (복잡한 설정)  
- Heroku (유료 전환)  

In [7]:
print("\n" + "="*80)
print("Cell 22: 배포 파일 생성 - Procfile")
print("="*80)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Procfile (Railway 실행 명령어)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

procfile_content = """web: uvicorn app.main:app --host 0.0.0.0 --port $PORT
"""

(PROJECT_ROOT / "Procfile").write_text(procfile_content.strip())

print("✅ Procfile 생성")
print(procfile_content)

print("""
[설명]
  - web: Railway의 웹 서비스 타입
  - uvicorn app.main:app: FastAPI 앱 실행
  - --host 0.0.0.0: 외부 접속 허용
  - --port $PORT: Railway가 자동으로 포트 할당
""")


Cell 22: 배포 파일 생성 - Procfile
✅ Procfile 생성
web: uvicorn app.main:app --host 0.0.0.0 --port $PORT


[설명]
  - web: Railway의 웹 서비스 타입
  - uvicorn app.main:app: FastAPI 앱 실행
  - --host 0.0.0.0: 외부 접속 허용
  - --port $PORT: Railway가 자동으로 포트 할당



In [ ]:
print("\n" + "="*80)
print("Cell 23: 배포 파일 생성 - railway.json")
print("="*80)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# railway.json (Railway 빌드 설정)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

railway_json = '''{
  "$schema": "https://railway.app/railway.schema.json",
  "build": {
    "builder": "NIXPACKS"
  },
  "deploy": {
    "startCommand": "uvicorn app.main:app --host 0.0.0.0 --port $PORT",
    "restartPolicyType": "ON_FAILURE",
    "restartPolicyMaxRetries": 10
  }
}
'''

(PROJECT_ROOT / "railway.json").write_text(railway_json)

print("✅ railway.json 생성")
print(railway_json)

print("""
[설명]
  - builder: NIXPACKS (Railway 기본 빌더)
  - startCommand: 서버 시작 명령어
  - restartPolicyType: 실패 시 재시작
  - restartPolicyMaxRetries: 최대 10회 재시도
""")

In [ ]:
print("\n" + "="*80)
print("Cell 24: 배포 파일 생성 - nixpacks.toml")
print("="*80)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# nixpacks.toml (Poetry 의존성 설치)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

nixpacks_toml = '''[phases.setup]
nixPkgs = ["python311", "poetry"]

[phases.install]
cmds = [
  "poetry config virtualenvs.in-project true",
  "poetry install --no-dev --no-interaction --no-ansi"
]

[phases.build]
cmds = []

[start]
cmd = "uvicorn app.main:app --host 0.0.0.0 --port $PORT"
'''

(PROJECT_ROOT / "nixpacks.toml").write_text(nixpacks_toml)

print("✅ nixpacks.toml 생성")
print(nixpacks_toml)

print("""
[설명]
  - setup: Python 3.11 + Poetry 설치
  - install: Poetry로 의존성 설치
  - --no-dev: 개발 의존성 제외 (빠른 배포)
  - start: 서버 시작
""")

In [8]:
print("\n" + "="*80)
print("Cell 25: .gitignore 업데이트")
print("="*80)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# .gitignore 확인/업데이트
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

gitignore_content = """# Python
__pycache__/
*.py[cod]
*$py.class
*.so
.Python

# 가상환경
env/
venv/
ENV/
.venv/

# Poetry
poetry.lock

# 환경변수 (중요!)
.env

# IDE
.vscode/
.idea/
*.swp
*.swo

# 운영체제
.DS_Store
Thumbs.db

# 프로젝트
runs/
*.log

# Jupyter
.ipynb_checkpoints/
"""

(PROJECT_ROOT / ".gitignore").write_text(gitignore_content.strip())

print("✅ .gitignore 업데이트")

print("""
⚠️ 중요: .env 파일이 .gitignore에 포함되어야 합니다!
  → API 키가 GitHub에 올라가면 안 됩니다.
  → Railway에서 환경변수를 별도로 설정합니다.
""")


Cell 25: .gitignore 업데이트
✅ .gitignore 업데이트

⚠️ 중요: .env 파일이 .gitignore에 포함되어야 합니다!
  → API 키가 GitHub에 올라가면 안 됩니다.
  → Railway에서 환경변수를 별도로 설정합니다.

